In [1]:
# potrzebne bilioteki
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input,State
from dash_bootstrap_components._components.Container import Container
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [2]:
# wczytanie danych
df=pd.read_excel(r'C:\Users\natal\OneDrive\Dokumenty\AnalizaDanych\DATA\1.1.Wypadki_ogolem_smiertleni_ranni_POW.xlsx')
df_Wypadki=df.groupby(["Jednostka", "Rok"], as_index = False).sum()
# map emotions to a color
c = dict(zip(df_Wypadki["Jednostka"].unique(), px.colors.qualitative.G10))

In [3]:
external_stylesheets = [dbc.themes.FLATLY]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [4]:
#dodanie navbar
navbar = dbc.NavbarSimple(
    brand="Analiza i prognozowanie wypadków drogowych w Polsce na podstawie danych za lata 2014-2020",
    brand_href="/",
    color="primary",
    dark=True,
)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    navbar,
    html.Div(id='page-content')
])

#zdefiniowanie styli
cardStyle = {'minHeight': '180px', 'display': 'flex', 'justifyContent': 'space-between'}
headerStyle = {'margin': 'auto'}


index_page = html.Div([
    dbc.Container([
        dbc.Row([
            dbc.Col(html.H1("Projekt na kierunku Data Scientist.Analityk danych", className="text-center")
                    , className="mb-5 mt-5")
        ]),
        dbc.Row([
            dbc.Col(dbc.Card(children=[html.H3(children='Wizualizacja wypadków ogółem',
                                               className="text-center",
                                              style=headerStyle),
                                       dbc.Button("Kliknij",
                                                  href='/',
                                                  color="primary",
                                                  className="mt-3"),
                                      ],
                             body=True, color="dark", outline=True, style=cardStyle)
                    ,width = 4, className="mb-4"),

            dbc.Col(dbc.Card(children=[html.H3(children='Wizualizacja wybranych przyczyn',
                                               className="text-center",
                                              style=headerStyle),
                                       dbc.Button("Kliknij",
                                                  href='/page-1',
                                                  color="primary",
                                                  className="mt-3"),
                                       ],
                             body=True, color="dark", outline=True, style=cardStyle)
                    ,width = 4,className="mb-4"),

            dbc.Col(dbc.Card(children=[html.H3(children='Prognoza',
                                               className="text-center",
                                                style=headerStyle),
                                       dbc.Button("Kliknij",
                                                  href='/page-2',
                                                  color="primary",
                                                  className="mt-3"),

                                       ],
                             body=True, color="dark", outline=True, style=cardStyle)
                    ,width =4, className="mb-4"),
        ]),
         dbc.Row([
            dbc.Col(html.H5(children='Do analizy wykorzystano Dash by Plotly.'
                                     )
                    , className="mb-4")
            ]),

        dbc.Row([
            dbc.Col(html.H5(children='Po kliknięciu na odpowiednią kartę można przejść do wizualizacji.')
                    , className="mb-5")
        ]),
        
    ]),
])
        

page_1_layout = html.Div([
        html.H1('Analiza wypadków samochodowych w Polsce w latach 2014-2020'),
        html.Div([
        dcc.Dropdown(id='dpdn2', value=df_Wypadki['Jednostka'].unique(), multi=True,
                     options=[{'label': x, 'value': x} for x in
                              df_Wypadki.Jednostka.unique()]),
        html.Div([
            dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None,  
                      config={
                          'staticPlot': False,     
                          'scrollZoom': True,     
                          'doubleClick': 'reset',  
                          'showTips': False,       
                          'displayModeBar': True,  
                          'watermark': True,},
                      className='' #six columns
                      ),
            #dcc.Graph(id='pie-graph', figure={}, className='six columns'),
            #dcc.Graph(id='pie-graph2', figure={}, className='six columns'),
        ]),
        html.Div([dcc.Graph(id='pie-graph', figure={}, className='six columns'),#six columns
            dcc.Graph(id='pie-graph2', figure={}, className='six columns')]) #six columns
    ]), 
    html.Div(id='page-1-content'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

page_2_layout = html.Div([
    html.H1('Page 2'),
    html.Div(id='page-2-content'),
    html.Br(),
    dcc.Link('Analiza wypadków', href='/page-1'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

# Update the index
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page
    # You could also return a 404 "URL not found" page here

In [5]:
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value')
)

def update_graph(Jednostka_chosen):
    dff = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
    fig = px.line(data_frame=dff, x='Rok', y='Wypadki ogółem', color='Jednostka',
                  custom_data=['Jednostka', 'Wypadki ogółem', 'Ofiary śmiertelne', 'Ranni'],
                 title=f'Wypadki ogółem: ', color_discrete_map=c)
    fig.update_traces(mode='lines+markers')
    return fig



In [6]:
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title='Ofiary śmiertelne w 2014',color="Jednostka", color_discrete_map=c)
        return fig2
    else:
        print(f'hover data: {hov_data}')
        #print(hov_data['points'][0]['customdata'][0])
        #print(f'click data: {clk_data}')
        #print(f'selected data: {slct_data}')
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title=f'Ofiary śmiertelne w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
    
@app.callback(
    Output(component_id='pie-graph2', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title='Ranni w 2014',color='Jednostka', color_discrete_map=c)
        return fig2
    else:
        print(f'hover data: {hov_data}')
        #print(hov_data['points'][0]['customdata'][0])
        #print(f'click data: {clk_data}')
        #print(f'selected data: {slct_data}')
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title=f'Ranni w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
        


In [ ]:

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Dec/2021 21:04:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 21:04:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 21:04:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 21:04:37] "POST /_dash-update-component HTTP/1.1" 200 -
